In [23]:
import pandas as pd 

In [24]:
data=pd.read_csv("D:\KaggleCompetitions\Email_Spam\email.csv")
data.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5573 entries, 0 to 5572
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5573 non-null   object
 1   Message   5573 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [26]:
spam_cat=data[['Category']]
print(type(spam_cat))

<class 'pandas.core.frame.DataFrame'>


In [27]:
# from sklearn.preprocessing import OrdinalEncoder

# ordinal_enco=OrdinalEncoder()
# spam_cat_enco=ordinal_enco.fit_transform(spam_cat)
# spam_cat_enco.dtype

In [28]:
# spam_new=pd.DataFrame(spam_cat_enco,columns=data.columns,index=data.index)

In [29]:
Mail=data['Message'].astype(str)
target=data['Category']

In [30]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(Mail,target,test_size=0.2,random_state=42)

In [31]:
from sklearn.linear_model import SGDClassifier

# sgd_clf=SGDClassifier(loss='log_loss',random_state=42)
# sgd_clf.fit(x_train,y_train)

In [32]:
# from sklearn.preprocessing import OneHotEncoder

# one_hot_enco=OneHotEncoder()

# Mail_encoded=one_hot_enco.fit_transform(Mail)


In [33]:
# Mail_encoded

In [34]:
# target_encoded=one_hot_enco.fit_transform(target)

In [35]:
# target_encoded

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

pipeline=Pipeline([
    ('tfidf',TfidfVectorizer(
        ngram_range=(1,2),
        stop_words='english',
        max_df=0.9
    )),
    ('sgd',SGDClassifier(loss='log_loss',random_state=42))
])

pipeline.fit(x_train,y_train)

,steps,"[('tfidf', ...), ('sgd', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [37]:
y_prob=pipeline.predict_proba(x_test)
y_pred=(y_prob[:1]>0.45).astype(int)
y_pred

array([[1, 0, 0]])

In [ ]:
#threshold increasing 
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [1115, 1]

## Tuning : Character n-grams 

In [40]:
pipeline_char = Pipeline([
    ('tfidf', TfidfVectorizer(
        analyzer='char',
        ngram_range=(3,5),
        min_df=3
    )),
    ('sgd', SGDClassifier(loss='log_loss', random_state=42))
])

pipeline_char.fit(x_train, y_train)
y_pred_char = pipeline_char.predict(x_test)


In [ ]:
y_pred=pipeline.predict(x_test)

In [41]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_char))
print(classification_report(y_test, y_pred_char))

[[958   0]
 [ 24 133]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       958
        spam       1.00      0.85      0.92       157

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



## Tuning 3 : Class weighting 

In [42]:
pipeline_weighted = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1,2),
        stop_words='english',
        max_df=0.9
    )),
    ('sgd', SGDClassifier(
        loss='log_loss',
        class_weight='balanced',
        random_state=42
    ))
])

pipeline_weighted.fit(x_train, y_train)
y_pred_weighted = pipeline_weighted.predict(x_test)


In [43]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_weighted))
print(classification_report(y_test, y_pred_weighted))

[[952   6]
 [ 12 145]]
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       958
        spam       0.96      0.92      0.94       157

    accuracy                           0.98      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [45]:
pipeline_alpha = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1,2),
        stop_words='english',
        max_df=0.9
    )),
    ('sgd', SGDClassifier(
        loss='log_loss',
        alpha=1e-4,   # try 1e-3, 1e-5 later
        random_state=42
    ))
])

pipeline_alpha.fit(x_train, y_train)
y_pred_alpha = pipeline_alpha.predict(x_test)


In [46]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_alpha))
print(classification_report(y_test, y_pred_alpha))

[[956   2]
 [ 21 136]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       958
        spam       0.99      0.87      0.92       157

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [44]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__max_df': [0.8, 0.9],
    'sgd__alpha': [1e-4, 1e-3]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1
)

grid.fit(x_train, y_train)

best_model = grid.best_estimator_
y_pred_grid = best_model.predict(x_test)

print(grid.best_params_)


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]
  warnings.warn(


{'sgd__alpha': 0.0001, 'tfidf__max_df': 0.8, 'tfidf__ngram_range': (1, 1)}
